In the code cell below I import the python libraries that are used in this project. The dataset is loaded onto a pandas dataframe and df.head() is used to verify import path.

In [7]:
# Caffeine intake health prediction

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv('../data/synthetic_coffee_health_10000.csv')

display(df.head())
display(df.info())
display(df.describe())

,ID,Age,Gender,Country,Coffee_Intake,Caffeine_mg,Sleep_Hours,Sleep_Quality,BMI,Heart_Rate,Stress_Level,Physical_Activity_Hours,Health_Issues,Occupation,Smoking,Alcohol_Consumption
0,1,40,Male,Germany,3.5,328.1,7.5,Good,24.9,78,Low,14.5,NaN,Other,0,0
1,2,33,Male,Germany,1.0,94.1,6.2,Good,20.0,67,Low,11.0,NaN,Service,0,0
2,3,42,Male,Brazil,5.3,503.7,5.9,Fair,22.7,59,Medium,11.2,Mild,Office,0,0
3,4,53,Male,Germany,2.6,249.2,7.3,Good,24.7,71,Low,6.6,Mild,Other,0,0
4,5,32,Female,Spain,3.1,298.0,5.3,Fair,24.1,76,Medium,8.5,Mild,Student,0,1


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   ID                       10000 non-null  int64  
 1   Age                      10000 non-null  int64  
 2   Gender                   10000 non-null  object 
 3   Country                  10000 non-null  object 
 4   Coffee_Intake            10000 non-null  float64
 5   Caffeine_mg              10000 non-null  float64
 6   Sleep_Hours              10000 non-null  float64
 7   Sleep_Quality            10000 non-null  object 
 8   BMI                      10000 non-null  float64
 9   Heart_Rate               10000 non-null  int64  
 10  Stress_Level             10000 non-null  object 
 11  Physical_Activity_Hours  10000 non-null  float64
 12  Health_Issues            4059 non-null   object 
 13  Occupation               10000 non-null  object 
 14  Smoking                

None

,ID,Age,Coffee_Intake,Caffeine_mg,Sleep_Hours,BMI,Heart_Rate,Physical_Activity_Hours,Smoking,Alcohol_Consumption
count,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.00000,10000.000000
mean,5000.50000,34.949100,2.509230,238.411010,6.636220,23.986860,70.617800,7.48704,0.20040,0.300700
std,2886.89568,11.160939,1.450248,137.748815,1.222055,3.906411,9.822951,4.31518,0.40032,0.458585
min,1.00000,18.000000,0.000000,0.000000,3.000000,15.000000,50.000000,0.00000,0.00000,0.000000
25%,2500.75000,26.000000,1.500000,138.750000,5.800000,21.300000,64.000000,3.70000,0.00000,0.000000
50%,5000.50000,34.000000,2.500000,235.400000,6.600000,24.000000,71.000000,7.50000,0.00000,0.000000
75%,7500.25000,43.000000,3.500000,332.025000,7.500000,26.600000,77.000000,11.20000,0.00000,1.000000
max,10000.00000,80.000000,8.200000,780.300000,10.000000,38.200000,109.000000,15.00000,1.00000,1.000000


The next step is to check if we have any missing values in our dataset by running isnull().sum(). By doing this we can see that the chosen dataset is clean of missing values except for health issues where 59.41% of people are free from health issues. One thing to note with this is that we cant be sure if all 5941 are healthy or if some of these are actual missing values. As the column Health_Issues is a categorical variable None, Mild, Moderate, Severe I am replacing the NaN's with string None to match the description of the dataset on Kaggle and to make it more consistent for modelling and analysis. I also dropped the column Coffee_Intake as it just contains how many cups of coffee individuals have but since we also have Caffeine_mg it is just redundant. Also it's not an accurate way to measure caffeine intake as I'm not sure if things like energy drinks are counted in it. Also check rows and number of columns

In [9]:
df_missing = pd.DataFrame(df)

df_missing = df_missing.drop(columns=['Coffee_Intake'])

display(df_missing.isnull().sum())

df_missing['Health_Issues'] = df_missing['Health_Issues'].fillna('None')
display(df_missing['Health_Issues'])

df_missing.shape

ID                            0
Age                           0
Gender                        0
Country                       0
Caffeine_mg                   0
Sleep_Hours                   0
Sleep_Quality                 0
BMI                           0
Heart_Rate                    0
Stress_Level                  0
Physical_Activity_Hours       0
Health_Issues              5941
Occupation                    0
Smoking                       0
Alcohol_Consumption           0
dtype: int64

0           None
1           None
2           Mild
3           Mild
4           Mild
          ...   
9995    Moderate
9996        Mild
9997        None
9998        None
9999        None
Name: Health_Issues, Length: 10000, dtype: object

(10000, 15)

The next step I took was to look at my data and how things are distributed and balanced. One thing I noticed right away was that the category for severe health problems only contain 17 individuals out of 10 000 (0.0017%) which is extremely rare so to make predictions with a model "better" I combined Moderate and Severe into a new column called ModerateAndSevere.

In [ ]:
for col in ['Health_Issues', 'Gender', 'Stress_Level', 'Smoking', 'Alcohol_Consumption']:
    print(df_missing[col].value_counts())
    print()

df_colcombined = df_missing.copy()
df_colcombined['ModerateAndSevere'] = np.where(df_colcombined['Health_Issues'].isin(['Moderate', 'Severe']), 'ModerateAndSevere', df_colcombined['Health_Issues'])


Health_Issues
None        5941
Mild        3579
Moderate     463
Severe        17
Name: count, dtype: int64

Gender
Female    5001
Male      4773
Other      226
Name: count, dtype: int64

Stress_Level
Low       6989
Medium    2050
High       961
Name: count, dtype: int64

Smoking
0    7996
1    2004
Name: count, dtype: int64

Alcohol_Consumption
0    6993
1    3007
Name: count, dtype: int64

